# EDA & Explainability (lightweight)
Use feature importances from Gradient Boosting; SHAP optional.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
assets = pd.read_csv('asset_registry.csv'); pred = pd.read_csv('predictions_gas_corrosion.csv')
df = assets.merge(pred, on='AssetID', how='left'); gas = df[df['Commodity']=='Gas']
X_cols=['Region','System','AssetType','Material','CoatingType','SeamType','ClassLocation','HCA_Flag','SoilCorrosivityIndex','MAOP_psig','PipeToSoilPotential_mV','AgeYears','DiameterInches','CPStatus','CorrosionRiskScore_Rubric']
X=gas[X_cols]; y=(gas['PredictedRisk_Bucket'].isin(['High','Critical'])).astype(int)
pre=ColumnTransformer([('num',Pipeline([('imp',SimpleImputer(strategy='median'))]),['SoilCorrosivityIndex','MAOP_psig','PipeToSoilPotential_mV','AgeYears','DiameterInches','CorrosionRiskScore_Rubric','ClassLocation']),('cat',Pipeline([('imp',SimpleImputer(strategy='most_frequent')),('oh',OneHotEncoder(handle_unknown='ignore'))]),['Region','System','AssetType','Material','CoatingType','SeamType','HCA_Flag','CPStatus'])])
model=Pipeline([('pre',pre),('clf',GradientBoostingClassifier(random_state=42))]).fit(X,y)
importances=model.named_steps['clf'].feature_importances_
print('Top importances (first 10):',importances[:10])